<a href="https://colab.research.google.com/github/DHIVYASRI-D/medicinal-plant-classifier-and-insight-generator/blob/main/2_codegen_350M_multi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Install Dependencies (if fresh notebook)

In [ ]:
!pip install transformers datasets evaluate accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━ 244.9/664.8 MB 7.4 MB/s eta 0:00:57

2. Load and Preprocess Dataset

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

# Load Python subset from CodeSearchNet
dataset = load_dataset("code_search_net", "python")
tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen-350M-multi")
tokenizer.pad_token = tokenizer.eos_token

def tokenize(example):
    code_line = " ".join(example["func_code_tokens"])
    return tokenizer(code_line, truncation=True, max_length=128, padding="max_length")

tokenized_dataset = dataset.map(tokenize, batched=False)
tokenized_dataset = tokenized_dataset.map(lambda x: {"labels": x["input_ids"]})


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/12.9k [00:00<?, ?B/s]

code_search_net.py:   0%|          | 0.00/8.44k [00:00<?, ?B/s]

The repository for code_search_net contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/code_search_net.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


python.zip:   0%|          | 0.00/941M [00:00<?, ?B/s]

...

Map:   0%|          | 0/22176 [00:00<?, ? examples/s]

Map:   0%|          | 0/23107 [00:00<?, ? examples/s]

Map:   0%|          | 0/412178 [00:00<?, ? examples/s]

Map:   0%|          | 0/22176 [00:00<?, ? examples/s]

Map:   0%|          | 0/23107 [00:00<?, ? examples/s]

3. Load Model

In [ ]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("Salesforce/codegen-350M-multi")
model.resize_token_embeddings(len(tokenizer))


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/797M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/797M [00:00<?, ?B/s]

Some weights of the model checkpoint at Salesforce/codegen-350M-multi were not used when initializing CodeGenForCausalLM: ['transformer.h.0.attn.causal_mask', 'transformer.h.1.attn.causal_mask', 'transformer.h.10.attn.causal_mask', 'transformer.h.11.attn.causal_mask', 'transformer.h.12.attn.causal_mask', 'transformer.h.13.attn.causal_mask', 'transformer.h.14.attn.causal_mask', 'transformer.h.15.attn.causal_mask', 'transformer.h.16.attn.causal_mask', 'transformer.h.17.attn.causal_mask', 'transformer.h.18.attn.causal_mask', 'transformer.h.19.attn.causal_mask', 'transformer.h.2.attn.causal_mask', 'transformer.h.3.attn.causal_mask', 'transformer.h.4.attn.causal_mask', 'transformer.h.5.attn.causal_mask', 'transformer.h.6.attn.causal_mask', 'transformer.h.7.attn.causal_mask', 'transformer.h.8.attn.causal_mask', 'transformer.h.9.attn.causal_mask']
- This IS expected if you are initializing CodeGenForCausalLM from the checkpoint of a model trained on another task or with another architecture (

Embedding(50295, 1024)

4. Define Training Arguments and Trainer

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./codegen-350M-results",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=100,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"].select(range(1000)),
    eval_dataset=tokenized_dataset["validation"].select(range(200)),
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


5. Train It

In [ ]:
trainer.train()


Epoch,Training Loss,Validation Loss
1,1.189300,1.478269


TrainOutput(global_step=500, training_loss=1.3852479400634765, metrics={'train_runtime': 179.0666, 'train_samples_per_second': 5.585, 'train_steps_per_second': 2.792, 'total_flos': 232977425664000.0, 'train_loss': 1.3852479400634765, 'epoch': 1.0})

###Save the model

In [ ]:
model.save_pretrained("./codegen-350M-multi-finetuned")
tokenizer.save_pretrained("./codegen-350M-multi-finetuned")


('./codegen-350M-multi-finetuned/tokenizer_config.json',
 './codegen-350M-multi-finetuned/special_tokens_map.json',
 './codegen-350M-multi-finetuned/vocab.json',
 './codegen-350M-multi-finetuned/merges.txt',
 './codegen-350M-multi-finetuned/added_tokens.json',
 './codegen-350M-multi-finetuned/tokenizer.json')

Then download it or push to Google Drive:

In [ ]:
!cp -r ./codegen-350M-multi-finetuned /content/drive/MyDrive/finetuned-models/


Next time you restart Colab, you can load directly:

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/finetuned-models/codegen-350M-multi-finetuned")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/finetuned-models/codegen-350M-multi-finetuned")


In [ ]:
import torch
import math

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

def evaluate_accuracy_and_perplexity(model, tokenizer, dataset, max_samples=200):
    correct = 0
    total = 0
    losses = []

    for sample in dataset.select(range(max_samples)):
        inputs = torch.tensor(sample["input_ids"]).unsqueeze(0).to(device)
        labels = torch.tensor(sample["labels"]).unsqueeze(0).to(device)

        with torch.no_grad():
            outputs = model(input_ids=inputs, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

        losses.append(loss.item())
        predictions = torch.argmax(logits, dim=-1)
        mask = labels != -100
        correct += (predictions[mask] == labels[mask]).sum().item()
        total += mask.sum().item()

    accuracy = correct / total
    perplexity = math.exp(sum(losses) / len(losses))
    return accuracy, perplexity

accuracy, perplexity = evaluate_accuracy_and_perplexity(model, tokenizer, tokenized_dataset["validation"])
print(f"Accuracy: {accuracy:.4f}")
print(f"Perplexity: {perplexity:.4f}")


Accuracy: 0.2265
Perplexity: 4.3869
